# Report

## Introduction and data

> REMOVE THE FOLLOWING TEXT

This section includes an introduction to the project motivation, data, and research question.
Describe the data and definitions of key variables.

It should also include some exploratory data analysis.

All of the EDA won't fit in the paper, so focus on the EDA for the response variable and a few other interesting variables and relationships.

## Die Bedeutung von YouTube für Nachrichtenmedien

YouTube hat sich in den letzten Jahren zu einer der wichtigsten Plattformen für den Konsum von Videoinhalten entwickelt. Insbesondere für Nachrichtenmedien wie ntv ist die Plattform zu einem unverzichtbaren Kanal geworden, um ihre Zielgruppe zu erreichen. Die Analyse der Performance von YouTube-Videos und das Verständnis der Faktoren, die zum Erfolg einzelner Beiträge führen, sind dabei von entscheidender Bedeutung für die strategische Ausrichtung des Kanals.

Für Nachrichtenmedien stellt sich dabei die besondere Herausforderung, dass sie einerseits journalistischen Qualitätsstandards gerecht werden müssen, andererseits aber auch die Mechanismen der Plattform verstehen und nutzen müssen, um ihre Inhalte erfolgreich zu platzieren. Die vorliegende Analyse soll dazu beitragen, diese Balance besser zu verstehen und zu optimieren.

## Forschungsfrage und Hypothesen

Die zentrale Forschungsfrage dieser Analyse lautet: Welche Faktoren beeinflussen die Performance von Videos auf dem ntv YouTube-Kanal und wie können diese Erkenntnisse genutzt werden, um die Content-Strategie zu optimieren?

Dabei werden drei zentrale Hypothesen untersucht: Erstens wird angenommen, dass Videos zu bestimmten Themengebieten wie dem Ukrainekrieg oder Bitcoin im Durchschnitt mehr Aufrufe und eine längere Wiedergabedauer erzielen als Videos zu anderen Themen. Zweitens wird vermutet, dass die Art der Thumbnail-Gestaltung, insbesondere die prominente Platzierung bekannter Experten, einen signifikanten Einfluss auf die Klickrate hat. Die dritte Hypothese besagt, dass eine höhere SEO-Bewertung des Videotitels positiv mit der Anzahl der Aufrufe und der Wiedergabedauer korreliert.

## Datengrundlage und Erhebungsprozess

Der analysierte Datensatz umfasst etwa 2500 Videos, die im Jahr 2024 auf dem ntv YouTube-Kanal veröffentlicht wurden. Die Datenerhebung gestaltete sich dabei als komplexer Prozess, der verschiedene Quellen und Methoden kombiniert. Während grundlegende Metriken wie Aufrufe, Likes und Kommentare über die YouTube Analytics API bezogen werden können, sind spezielle Metriken wie die Klickrate der Impressionen nur durch manuelle Extraktion aus dem YouTube Studio Backend verfügbar. Dies macht die Datensammlung zu einem aufwändigen Prozess, der sowohl automatisierte als auch manuelle Schritte erfordert.

Zusätzlich zu den von YouTube bereitgestellten Metriken wurden weitere Kategorisierungen und Bewertungen vorgenommen. Die Videos wurden nach ihren Themenbereichen kategorisiert (Politik, Wirtschaft, Krieg, Sonstiges, Bilder, Live). Die Thumbnail-Gestaltung wurde nach einem dreistufigen System bewertet, das insbesondere die Präsenz und Positionierung wichtiger Persönlichkeiten berücksichtigt. Die Kategorie "3" gab es bei Thumbnails die eine Kombination von Gesicht und Titel besonders präsent ist. Die Kategorie "2" gab es, wenn eines von beiden vorhanden war. Entweder ein normales Bild + präsenten Tiel oder präsentes Gesicht. Eine "1" wurde für Thumbnails vergeben, die keine besonderen Merkmale aufgewiesen haben. In der Datei "youtube Daten abrufen Test2.ipynb" ist auch ein Code abgelegt, der über "Google Vision AI" eine Bewertung der Thumnbails nach diesen Kategorien erfolgen soll. Leider war hier die Parameter zu ungenau. Eine Verbesserung des systems wäre für das Projekt zu aufwendig und teuer gewesen. Hier müsste noch mehr Aufwand betrieben werden oder ein eigenes System trainiert werden. Das wäre die Aufgabe für ein neues Projekt. 

Für die Bewertung der Videotitel wurde ein KI-gestütztes System entwickelt, das die Titel nach SEO-Kriterien auf einer Skala von 0.0 bis 10,0 bewertet. Dabei wurden durch die KI Bewertungen zwischen 4,5 und 8,5 vorgenommen. Als KI wurde ChatGPT per API genutzt. Der Code und auch der Prompt dafür ist in der Datei youtube Daten abrufen Test2.ipynb abgelegt. 

## Zentrale Metriken und ihre Bedeutung für einen Nachrichtenkanal

Für die Analyse wurden drei zentrale Zielvariablen identifiziert, die für Nachrichtenmedien auf YouTube von besonderer Bedeutung sind:

Die durchschnittliche Wiedergabedauer ist besonders für Nachrichtenformate relevant, da sie Aufschluss darüber gibt, wie gut es gelingt, komplexe Sachverhalte für die YouTube-Zielgruppe aufzubereiten. Anders als bei traditionellen TV-Nachrichten, wo Zuschauer tendenziell das gesamte Format konsumieren, ist die Haltedauer auf YouTube ein kritischer Erfolgsfaktor. Sie beeinflusst direkt, wie häufig ein Video vom Algorithmus anderen Nutzern vorgeschlagen wird. Für Nachrichtenmedien ist dies eine besondere Herausforderung, da sie oft tagesaktuelle, erklärungsbedürftige Themen in einem Format präsentieren müssen, das die Aufmerksamkeit der Zuschauer fesselt.

Die Anzahl der Aufrufe ist für Nachrichtenkanäle nicht nur eine reine Reichweitenkennzahl, sondern auch ein Indikator für die gesellschaftliche Relevanz ihrer Berichterstattung. Im hart umkämpften Nachrichtenmarkt auf YouTube, wo etablierte Medien mit einer Vielzahl alternativer Informationsquellen konkurrieren, sind hohe Aufrufzahlen zudem entscheidend für die Monetarisierung und damit die Finanzierung hochwertiger journalistischer Arbeit. Besonders bei Breaking News und wichtigen politischen Ereignissen ist die schnelle Generierung hoher Aufrufzahlen ein wichtiger Wettbewerbsfaktor.

Die Klickrate der Impressionen hat für Nachrichtenmedien eine besondere strategische Bedeutung. Sie zeigt, wie gut es gelingt, in der Informationsflut von YouTube die Aufmerksamkeit potenzieller Zuschauer zu gewinnen, ohne dabei in reißerische oder irreführende Darstellungen zu verfallen. Die Balance zwischen attraktiver Präsentation und journalistischer Seriosität ist dabei eine zentrale Herausforderung. Eine hohe Klickrate bei gleichzeitig guter Haltedauer deutet darauf hin, dass die Erwartungen der Zuschauer erfüllt wurden und keine "Clickbait"-Strategie verfolgt wird.

Zusammen bilden diese drei Metriken ein komplexes Erfolgssystem: Während die Klickrate die initiale Attraktivität eines Nachrichtenvideos misst, zeigt die Wiedergabedauer, ob der Inhalt die geweckten Erwartungen erfüllt. Die Gesamtaufrufe sind schließlich das Resultat dieser beiden Faktoren in Kombination mit der algorithmischen Verbreitung durch YouTube. Für Nachrichtenmedien ist das Verständnis dieser Zusammenhänge essentiell, um ihre journalistischen Inhalte erfolgreich auf der Plattform zu platzieren.

## Limitationen und Herausforderungen

Bei der Analyse müssen einige wichtige Einschränkungen berücksichtigt werden. Die teilweise notwendige manuelle Datenextraktion erhöht das Risiko von Fehlern und macht den Prozess zeitaufwändig. Die Kategorisierung der Themen und die Bewertung der Thumbnails enthalten zwangsläufig subjektive Elemente, auch wenn versucht wurde, diese durch klare Kriterien zu minimieren. Zudem spielen externe Faktoren wie aktuelle Nachrichtenereignisse, Änderungen im YouTube-Algorithmus und saisonale Effekte eine wichtige Rolle, die bei der Interpretation der Ergebnisse berücksichtigt werden müssen.

## Methodology

> REMOVE THE FOLLOWING TEXT

This section includes a brief description of your modeling process.

Explain the reasoning for the type of model you're fitting, predictor variables considered for the model.

Additionally, show how you arrived at the final model by describing the model selection process, variable transformations (if needed), assessment of conditions and diagnostics, and any other relevant considerations that were part of the model fitting process.

## Results

> REMOVE THE FOLLOWING TEXT

This is where you will output the final model with any relevant model fit statistics.

Describe the key results from the model.
The goal is not to interpret every single variable in the model but rather to show that you are proficient in using the model output to address the research questions, using the interpretations to support your conclusions.

Focus on the variables that help you answer the research question and that provide relevant context for the reader.


## Discussion + Conclusion


> REMOVE THE FOLLOWING TEXT

In this section you'll include a summary of what you have learned about your research question along with statistical arguments supporting your conclusions.
In addition, discuss the limitations of your analysis and provide suggestions on ways the analysis could be improved.
Any potential issues pertaining to the reliability and validity of your data and appropriateness of the statistical analysis should also be discussed here.
Lastly, this section will include ideas for future work.